## Detect languages used in bulletins

In [2]:

import os
from collections import defaultdict

import langid
import pycountry

Le code ci-dessous permet de forcer l'algorithme à choisir entre le français ou le néerlandais


In [ ]:
langid.set_languages(['fr', 'nl'])

Nous listons tous les fichiers .txt dans le corpus



In [3]:
root = "../data/txt/"
txts = os.listdir(root)
print(f"{len(txts)} TXT files found")

Number of articles: 18846


Nous allons lire chaque fichier, détecter la langue, et incrémenter lang_dictlorsqu'une langue est détectée.

Important : pour détecter les langues sur tous les documents, mettez limit = None ci-dessous.

In [ ]:
limit = 500
# limit = None

In [5]:
lang_dict = defaultdict(int)
txts = txts[:limit] if limit else texts

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']


In [6]:
for i, txt in enumerate(sorted(txts)):
    if i % 50 == 0:
        print(f'{i} document(s) processed...')
    text = open(os.path.join(root, txt)).read()
    text_length = len(text)
    if text_length > 20:
        lang, conf = langid.classify(text)
        lang_dict[lang] += 1
    else:
        print(f"{txt} contains only {text_length} characters, treating as unknown")
        lang_dict['n/a'] += 1

===== rec.sport.hockey =====
From: Mamatha Devineni Ratnam <mr47+@andrew.cmu.edu> Subject: Pens fans reactions Organization: Post Office, Carnegie Mellon, Pittsburgh, PA Lines: 12 NNTP-Posting-Host: po4.andrew.cmu.edu    I am sure some bashers of Pens fans are pretty confused about the lack of any kind of posts about the recent Pens massacre of the Devils. Actually, I am  bit puzzled too and a bit relieved. However, I am going to put an end to non-PIttsburghers' relief with a bit of praise for the Pens. Man, they are killing those Devils worse than I thought. Jagr just showed you why he is much better than his regular season stats. He is also a lot fo fun to watch in the playoffs. Bowman should let JAgr have a lot of fun in the next couple of games since the Pens are going to beat the pulp out of Jersey anyway. I was very disappointed not to see the Islanders lose the final regular season game.          PENS RULE!!!   

===== comp.sys.ibm.pc.hardware =====
From: mblawson@midway.ecn.uok

Nous allons trier les langues de la plus fréquente à la plus rare.

In [ ]:
sorted_lang = sorted(lang_dict.items(), key=lambda kv: kv[1], reverse=True)

Et nous imprimons les résultats...



In [ ]:

for lang_code, nb_docs in sorted_lang:
    language = pycountry.languages.get(alpha_2=lang_code)
    try:
        lang_name = language.name
    except AttributeError:
        lang_name = language
    print(f"{lang_name}\t{nb_docs}")